# Complete Solution

This notebook that analyses tree point cloud.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

# Import modules.
import os
import glob
import pathlib
import numpy as np
import pandas as pd
import open3d as o3d
from tqdm import tqdm

from utils import (  # local source
      ahn_utils,
      las_utils,
      o3d_utils,
      tree_utils
  )

In [ ]:
# Setup logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

#### Process File
----

In [ ]:
adTree_exe = '../../bin/AdTree'

# Load point cloud data. source can be: ahn / cyclo / sonarski
las_file = '../../dataset/cyclo/filtered_tree_121913_487434.las'
tree_cloud = o3d_utils.read_las(las_file)

# surface reconstruction
ahn_data_folder = '../../dataset/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)
treecode = las_utils.get_treecode_from_filename(las_file)
ground_cloud = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(npz_reader.get_tree_surface(treecode)))

In [ ]:
o3d.visualization.draw_geometries([ground_cloud,tree_cloud])

In [ ]:
# Analyse tree
result = tree_utils.process_tree(tree_cloud, ground_cloud, adTree_exe)

In [ ]:
# generate lod
r = result[0]
lod_3 = tree_utils.generate_LOD_v3(r['DBH']/2, r['stem_basepoint'], r['crown_basepoint'], r['crown_mesh-convex'])
o3d_utils.show_mesh(lod_3)

#### Process Dataset
----

In [ ]:
dataset_folder = '../../dataset/'

In [ ]:
dataset_folder = pathlib.Path(dataset_folder)
ahn_data_folder = dataset_folder.joinpath('ahn_surf/')
npz_reader = ahn_utils.NPZReader(ahn_data_folder)

DATA_KEYS = ['source', 'treecode', 'stem_basepoint', 'tree_height', 'stem_height',
             'crown_height', 'crown_baseheight', 'stem_angle', 'DBH', 'CBH',
             'crown_shape', 'crown_diameter', 'crown_volume-alpha', 'crown_volume-convex']


In [ ]:
file_types = ('.LAS', '.las', '.LAZ', '.laz')
files = [f for f in dataset_folder.glob('*/filtered_tree_*')
             if f.name.endswith(file_types)]

results = []
for f in tqdm(files, unit="file",  smoothing=0):
    treecode = las_utils.get_treecode_from_filename(f.name)
    tree_cloud = o3d_utils.read_las(f)
    ground_cloud = npz_reader.get_surface(treecode)

    # process
    tree_data, _ = tree_utils.process_tree(tree_cloud, ground_cloud, adTree_exe)
    tree_data = {key:tree_data[key] for key in tree_data.keys() if key in DATA_KEYS}
    tree_data['source'] = f.parent.name
    tree_data['treecode'] = treecode

    results.append(tree_data)

df = pd.DataFrame(results)
df.to_csv(dataset_folder.joinpath('results.csv'))